> If you have not installed the necessary packages, please run:
> `conda install conda-forge::radgeetoolbox conda install conda-forge::geemap anaconda::ipykernel conda-forge::matplotlib`

In [1]:
import ee
from RadGEEToolbox import LandsatCollection, ExportToDrive, get_palette
import geemap

#### Authenticate

In [ ]:
# Store name of Google Cloud Project assosiated with Earth Engine - replace with your project ID/name
PROJECT_ID = 'your-project-id-here' #your-project-id-here shown on your Google Cloud Console Dashboard ⚠️⚠️⚠️
# Attempt to initialize Earth Engine
try:
    ee.Initialize(project=PROJECT_ID)
    print("Earth Engine initialized successfully.")
except Exception as e:
    print("Initialization failed, attempting authentication...")
    try:
        ee.Authenticate()
        ee.Initialize(project=PROJECT_ID)
        print("Authentication and initialization successful.")
    except Exception as auth_error:
        print("Authentication failed. Error details:", auth_error)

Earth Engine initialized successfully.


#### Define the image collection. In this case, a Landsat collection between 2013 through 2025 covering Salt Lake County in Utah, using images with 30% cloud cover or less.
#### Then, we create an NDVI collection and reduce the collection to a yearly mean collection, where we will have a mean NDVI image for each year between 2013 an 2025

In [3]:
# Define a region of interest using an ee.Geometry object
salt_lake_county_geometry = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('NAME', 'Salt Lake')).geometry()

# Create a LandsatCollection object with the specified parameters, filtered to the geometry
base_col = LandsatCollection(start_date='2013-01-01', end_date='2025-12-10', boundary=salt_lake_county_geometry, scale_bands=False, cloud_percentage_threshold=30)
# Compute NDVI for the collection
ndvi_col = base_col.ndvi
# Compute the yearly mean NDVI for the collection
yearly_ndvi = ndvi_col.yearly_mean_collection()


dates = yearly_ndvi.dates
print(dates)

['2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01', '2024-01-01', '2025-01-01']


#### Calculate the Mann-Kendall trend analysis and perform a Sen's Slope calculation using `mann_kendall_trend()` for the `ndvi` band within the study area

In [4]:
mann_kendall = yearly_ndvi.mann_kendall_trend(target_band='ndvi', join_method='system:time_start', geometry=salt_lake_county_geometry)

It's a good idea to query your result to ensure you get no errors. In this case, a good way to do this is by printing the band names of the resulting image as it forces GEE to run the calculation and it tells us the band names of the resulting image. 

In [5]:
print(mann_kendall.bandNames().getInfo())

# The s_statistic band contains the Mann-Kendall S statistic values
# the variance band contains the variance of the S statistic
# the z_score band contains the standardized test statistic (Z)
# the confidence band contains the probabilistic confidence level (0-1)
# the slope band contains the Sen's slope values

['s_statistic', 'variance', 'z_score', 'confidence', 'slope']


#### Optionally export the analysis to a geotiff raster within a Google Drive folder. In this case we are only going to export the 'slope' and 'confidence' bands

> NOTE: GEE will only export bands that are of a floating point type. The s_statistic band is of a LONG type (purely whole integers) so including that band without converting to a floating point will cause an error. We avoid this by only choosing bands that are already of floating point (values with decimals)

In [ ]:
want_to_export = True # Set to False if you do not want to export

if want_to_export:
    export = ExportToDrive(input_data=mann_kendall.select(['slope', 'confidence']),
                        description='Mann_Kendall_NDVI_Salt_Lake_County_2013_2025',
                        folder='GEE_Exports',
                        region=salt_lake_county_geometry,
                        crs='EPSG:4326',
                        scale=30,
                        maxPixels=1e13).export()

Starting export task for single image: Mann_Kendall_NDVI_Salt_Lake_County_2013_2025
Task ID: L2EGNHVIP4X65W25CMS5HOPH


#### Optionally visualize the results!

In [6]:
Map = geemap.Map(center=[40.751, -111.943], zoom=10)
Map.addLayer(mann_kendall, {'bands': ['confidence'], 'min': 0, 'max': 1, 'palette': get_palette('inferno')}, 'Probabalistic Confidence Level (%)')
Map.addLayer(mann_kendall, {'bands': ['slope'], 'min': -0.025, 'max': 0.025, 'palette': ['brown', 'white', 'green']}, 'Sen Slope')
Map

Map(center=[40.751, -111.943], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topri…